In [13]:
from oa_dag.utils import json_load

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    'mistralai/Mistral-7B-v0.3',
)
my_pad_token_id = tokenizer.encode('_')[-1]

In [11]:
DIR = '/share/edc/home/yuxi_xie/oa_dag/checkpoints/oa/metamath-mistral'
decoding_tracks = json_load(f'{DIR}/newscheme-mu0.05to0.55-dymin-rmu0.55-insert0.5-e3/decoding_tracks_step_denoise-zero3.json')

In [12]:
prompts, outputs = decoding_tracks['prompts'], decoding_tracks['outputs']
tracks = decoding_tracks['tracks']

In [17]:
tracks[0][0][0]

[1782]

In [5]:
idx = 0
print(outputs[idx])
for i in range(len(tracks[idx])):
    ids, pos = tracks[idx][i]
    start_idx, end_idx = min(pos), max(pos)
    print(tokenizer.decode([ids[pos.index(x)] if x in pos else my_pad_token_id for x in range(start_idx, end_idx + 1)]))
    # print(''.join([tokenizer.decode([decoding_tracks['tracks'][idx][i][0][decoding_tracks['tracks'][idx][i][1].index(x)]]) if x in decoding_tracks['tracks'][idx][i][1] else '_' for x in range(start_idx, end_idx + 1)]))
    # print(decoding_tracks['tracks'][idx][i])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
The Kennel house keeps 3 German Shepherds and 2 Bulldogs. If a German Shepherd consumes 5 kilograms of dog food and a bulldog consumes 3 kilograms of dog food per day. How many kilograms of dog food will they need in a week?

### Response:The number of kilograms of dog consumed per day is 3 x 5 = 15 kilograms.
The amount of dog food consumed per day per day is 2 x 3 = 6 kilograms.
Therefore, the total amount of dog food consumed in a day is 15 + 6 = 21 kilograms.
In a week, there are 7 days, so the total amount of dog food consumed in a week is 21 x 7 = 147 kilograms.
#### 147
The answer is: 14711
ior
ior"]ible
ior"]ible _ away
ior"]ible _ away _{
ior"]ible Du away _skinskin
ior"]ible Du away fict _ _ _ _ str str
 _ _ _ strЏu away fict{
 strЏ은ble Du away fict{
 strЏ은原le Du away fict{
skinЏ은原oweru away fict{
skinЏ은原ower —away fict{
skin strЏ은owerAsAsfict{
skinposЏ은